<a href="https://colab.research.google.com/github/oYasuiHaruki/Colaboratory/blob/main/%E7%8C%AE%E7%AB%8B%E8%87%AA%E5%8B%95%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

globを利用して、uploadされたexcelのfilepathを取得する。

In [155]:
import pandas as pd
from glob import glob

1つのexcelファイルを上げる前提。globでそのまま取得すると、リスト形式なので、インデックスで0を指定。

In [156]:
filepaths = glob('/content/*.xlsx')
filepath = filepaths[0]

複数シートがあるエクセルの場合は、read_excelのタイミングでsheet_nameの変数に指定する必要あり。  

指定しない場合はdefaultの0、つまり一枚目が入る。

In [157]:
kondate_hyo = pd.read_excel(filepath, sheet_name = "献立表")

In [140]:
recipe_detail = pd.read_excel(filepath, sheet_name = "レシピ詳細")

レシピ詳細から、材料と数量を取ってくるものを入れておくリスト。

In [141]:
zairyo_lists = []
suryo_lists = []

excelから分析に必要最小限のものを切り取る。これを先にしておかないと、あとで処理が大変。

In [142]:
kondate_hyo = kondate_hyo.iloc[2:8, 3:10]

列名を変更。

In [143]:
kondate_hyo.columns = ['日曜日', 
                       '月曜日',
                       '火曜日',
                       '水曜日',
                       '木曜日',
                       '金曜日',
                       '土曜日',
                       ]

ilocを利用するため、indexを修正。

In [144]:
kondate_hyo.reset_index(inplace = True, drop = True)

欠損値を文字列を代入することで、欠損値判定を楽にする。

In [145]:
kondate_hyo.fillna(value = "なし", inplace = True)



1.   献立表から料理を取得してくる。
2.   レシピ詳細から、1で取得した料理の材料と数量を取得してくる。
3.   zairyo_listsとsuryo_listsに入れる。
4.   繰り返す



In [ ]:
for n in range(7):
    for m in range(6):
        item = kondate_hyo.iloc[m, n]
        if item == 'なし':
            print('なし')
            continue
        print(f'itemは{item}')
        target_row = recipe_detail[recipe_detail['料理名'] == item]
        zairyo = target_row['材料'].values.tolist()[0]
        suryo = target_row['数量'].values.tolist()[0]
        zairyo_list = zairyo.split(',')
        suryo_list = suryo.split(',')
        for i in range(len(zairyo_list)):
            zairyo_lists.append(zairyo_list[i])
        for i in range(len(suryo_list)):
            suryo_lists.append(float(suryo_list[i]))
    print('\n')

dataframeにzairyo_listsとsuryo_listsを入れる。

In [147]:
todo_csv = pd.DataFrame(
    data = {'材料': zairyo_lists,
            '数量': suryo_lists}
)

In [148]:
todo_csv

,材料,数量
0,玉ねぎ,0.5
1,にんじん,0.3
2,豚ひき肉,130.0
3,玉ねぎ,0.5
4,豆腐,1.0
...,...,...
100,にんじん,0.3
101,豚ひき肉,130.0
102,玉ねぎ,0.5
103,豆腐,1.0


groupbyとlambdaを利用して、同じ料理名のものは数量を足し合わせる。

In [149]:
todo_csv = todo_csv.groupby('材料').apply(lambda x: x.sum()).drop('材料', axis = 1).reset_index()
todo_csv

,材料,数量
0,味噌,6.0
1,豆腐,12.0
2,にんじん,6.9
3,玉ねぎ,17.5
4,豚ひき肉,2990.0


csv形式で出力。 windowsのexcelだと文字化けしたので、文字コードをutf_8_sigに設定。

In [151]:
todo_csv.to_csv('todo.csv', encoding = 'utf_8_sig')